# Prediction with Decision Trees

In [2]:
## Importing all the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Data Gathering and Preprocessing

In [3]:
df = pd.read_csv("data_banknote_authentication.csv")  # loading the csv file

In [4]:
df.head()

,3.6216,8.6661,-2.8073,-0.44699,0
0,4.54590,8.1674,-2.4586,-1.46210,0
1,3.86600,-2.6383,1.9242,0.10645,0
2,3.45660,9.5228,-4.0112,-3.59440,0
3,0.32924,-4.4552,4.5718,-0.98880,0
4,4.36840,9.6718,-3.9606,-3.16250,0


In [5]:
df.columns

Index(['3.6216', '8.6661', '-2.8073', '-0.44699', '0'], dtype='object')

In [6]:
df[["0"]].value_counts()

0
0    761
1    610
Name: count, dtype: int64

In [7]:
## Train test split

train_set = df.sample(frac = 0.7)

test_set = df.drop(train_set.index)

train_set_arr = train_set.values
test_set_arr = test_set.values

new_train_X = train_set_arr[:,0:4]
new_train_y = train_set_arr[:,-1]

new_test_X = test_set_arr[:,0:4]
new_test_y = test_set_arr[:,-1]

## Calculate Gini index

In [8]:
## Implemnting the function for calculating gini index

def pi(l,m):
    
    no_of_rows = len(l)
    c = 0
    
    for i in l:
        
        if(i[-1] == m):
            c+=1

    return c/no_of_rows

def calc_gini_for_one_region(region,classes):
    
    c = 0

    for i in classes:
        p = pi(region,i)
        term = p**2
        c = c+term

    return 1-c

def gini_index(regions, classes):

    weighted_avg = 0

    for i in regions:
        n = len(i)
        if(n == 0):
            continue
        Q = calc_gini_for_one_region(i,classes)
        weighted_avg+= n*Q

    return weighted_avg


## Splitting the root node

In [9]:
def split_at_root_node(feature_index, value, dataset):
    
    L1 = []
    R1 = []
    
    for i in dataset:
        if (i[feature_index] < value):
            L1.append(i)
        else:
            R1.append(i)
    return L1, R1

## Evaluting the best split

In [10]:
def get_best_split(data):

    # Find the different classes present in the dataset
    class_values = [0,1]

    # initialize best_index, best_threshold, best_gini_score, best_regions
    optimum_loss = 2**31 - 1
    best_index 	= 2**31 - 1
    best_value 	= 2**31 - 1
    optimum_regions = None

    for i in range (len(data[0])-1):
        for j in data:

            focused_index = i
            focused_value = j[i]

            new_regions = split_at_root_node(focused_index,focused_value,data)  ## compute regions

            new_gini_score = gini_index(new_regions, class_values)  ## calculate gini score with newly computed regions
            

            if new_gini_score < optimum_loss:  # check if loss is getting decreased or not
                
                optimum_loss = new_gini_score
                optimum_regions = new_regions
                optimum_index = focused_index
                optimum_value = focused_value
                
    return {'index':optimum_index, 'value':optimum_value, 'regions':optimum_regions}

## Finally evaluting predictions at a Leaf node

In [11]:
def leaf_output(region):
    if not region:
        return None
    
    outcomes = list(map(lambda row: row[-1], region))
    
    if not outcomes:
        return None
    
    return max(set(outcomes), key=outcomes.count)

## Function for Recursive splitting

In [12]:
def split(node, max_depth, min_size, depth):
    
    left , right = node["regions"]  # evalue left and right regions of a given internal node
    
    if(depth >= max_depth):      # If depth is equal to max depth then evalute the predictions at the leaf node
        node["left"] = leaf_output(left)
        node["right"] = leaf_output(right)
        return
    
    if((len(left) <= min_size) or (len(right) <= min_size)):  # stopping criteria 
        node["left"] = leaf_output(left)
        node["right"] = leaf_output(right)
        return
    
    node['left'] = get_best_split(left)   # recursively splitting the left regions
    split(node['left'], max_depth, min_size, depth+1)
    
    node['right'] = get_best_split(right) # recursively splitting the right regions
    split(node['right'], max_depth, min_size, depth+1)
          

## Build the tree 

In [19]:
def build_tree(train_set,max_depth,min_size):
    root = get_best_split(train_set)
    split(root,max_depth,min_size,1)
    return root

In [20]:
tree = build_tree(train_set_arr, 5, 10)

## Predicting with the formed decision tree

In [21]:
## function for prediction
def predict(node, row):
    index = node['index']
    value = node['value']

    if row[index] < value:
        next_node = node['left']
    else:
        next_node = node['right']

    if isinstance(next_node, dict):
        return predict(next_node, row)
    else:
        return next_node

In [22]:
## Make predictions
arr= []
for i in new_test_X:
    
    arr.append(predict(tree,i))  ## predict for each i
    
v = np.array(arr)

## Calculate missclassification rate

In [25]:
## calculate the misclassification rate

# For testing set

correct = 0

for x,y in zip(v,new_test_y):
    if(x == y):
        correct+=1
        

misclassification_rate = (1-(correct/len(v)))*100
accuracy = (correct/len(v))*100

print(misclassification_rate/100)
print(accuracy)   

0.031630170316301665
96.83698296836984
